In [1]:
!pip install datasets
!pip install flash-attn --no-build-isolation
!pip install wandb
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which

In [ ]:
!wandb login

In [ ]:
from huggingface_hub import login
login()

In [ ]:
import wandb

wandb.init(
    project="gaokerena",
    name="instruction_tuning",
)

In [3]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainerCallback,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer, SFTConfig

BASE_MODEL_ID = "CohereForAI/aya-expanse-8b"
PRETRAINED_MODEL_ID = "gaokerena/pretrained"
DATASET_REPO = "gaokerena/MF3QA"
DATASET_SPLIT = "train"
WORKING_REPO_ID = "gaokerena/instruction_tuned"

HYPER_PARAMS = {
    "run_name": "pretraining",
    "output_dir": "outputs",
    "num_train_epochs": 1,
    "per_device_train_batch_size": 2,
    "gradient_accumulation_steps": 16,
    "optim": "adamw_torch",
    "logging_steps": 4,
    "save_strategy": "steps",
    "save_total_limit": 1,
    "learning_rate": 5e-4,
    "bf16": True,
    "max_grad_norm": 0.3,
    "warmup_ratio": 0.03,
    "lr_scheduler_type": "cosine",
    "weight_decay": 0.5,
    "report_to": "wandb",
    "gradient_checkpointing": True,
    "gradient_checkpointing_kwargs": {"use_reentrant": False},
    "hub_model_id": WORKING_REPO_ID,
    "dataloader_persistent_workers": True,
    "dataloader_num_workers": 4,
    "max_seq_length": 1024,
    "packing": False,
}

In [ ]:
from datasets import Dataset
dataset = load_dataset(DATASET_REPO, split=DATASET_SPLIT)
messages = {"messages": []}
for d in dataset:
  messages["messages"].append([
      {"content": d["Question"], "role": "user"},
      {"content": d["Answer"], "role": "assistant"},
  ])
dataset = Dataset.from_dict(messages)
dataset

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    torch_dtype=torch.bfloat16,
    device_map="cuda",
    low_cpu_mem_usage=True,
    attn_implementation="flash_attention_2"
)
model = PeftModel.from_pretrained(model, PRETRAINED_MODEL_ID)
model = model.merge_and_unload()
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
lora_config = LoraConfig(
    lora_alpha=2,
    lora_dropout=0.4,
    r=2,
    bias="none",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

In [ ]:
class PushToHubCallback(TrainerCallback):
    def on_save(self, args, state, control, **kwargs):
        kwargs["model"].push_to_hub(repo_id=WORKING_REPO_ID, commit_message=f"Checkpoint at step {state.global_step}")

In [ ]:
args = SFTConfig(
    **HYPER_PARAMS
)

trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset,
    peft_config=lora_config,
    callbacks=[PushToHubCallback],
)

In [ ]:
trainer.train()